# Introduction to Containerization with Docker

## What is containerization? What are Containers?

Containerization is the process of defining the software components required to run a *specific application*. In the context of a Docker-based containerization process this definition is written into a *Dockerfile*. 

A container is the executable software bundle that has been generated from a containerization process. In Docker the container is a *Docker Image*. 

### How is it different from virtualization (i.e. virtual machines)?

You may have encountered references to *virtual machines* (or VMs) as a model for running virtual computer environments within/on computer hardware. A common question that arises when discussing containerization is how it differs from virtualization.  The primary difference between the two is that the machine images and corresponding virtual machines that they enable are fully self-contained virtual systems that provide all of the core capabilities (in virtual form) of a dedicated computer, including a full operating system (such as Windows or Liunx [Mac OS is infrequently used as an OS for VMs]) and set of applications running within that operating system. Containerized apps, on the other hand share a common container engine that provides core operating system capabilities that allow the application code within the container to run *as if* it were operating within a fully functional OS and hardware environment. 

![Comparison of Hypervisor VM and Container Models](VMvsContainer.png)

## Why would I use containers in my work?

* Portability of **applications**
* Consistent behavior across systems
* Broad support
    * Cloud providers
        * [Google Cloud Platform](https://cloud.google.com/containers/)
        * [Amazon Elastic Container Service](https://aws.amazon.com/ecs/)
        * [Microsoft Azure Container Service](https://azure.microsoft.com/en-us/services/container-service/)
    * Desktop operating systems (Docker Engine & additional tools)
        * Mac OS
        * Windows
        * Linux
*  **You don't need to reinvent the wheel** - Many pre-configured containers have been developed and maintained by the community 
    * There are numerous container images and documentation maintained in [Docker Hub](https://hub.docker.com) and [Docker Store](https://store.docker.com/search?q=&source=community&type=image)
    * Many, many Dockerfiles maintained and accessible through [GitHub](https://github.com/search?utf8=✓&q=dockerfile&type=)
* Containers can provide a foundation for a reproducable environment & workflow
* Application containers are *imutable* and you can store data in data containers or through mounted directories on the host operating system

## Why would I *not* use containers?

* Your workflows and applications are primarily Windows or Mac OS application based
    * While you can run the Docker platform (i.e. Docker Engine) on a wide variety of operating systems, the applications that you can run in Docker containers are typically Linux based. 
* You typically work with applications for which there are robust installers for your particular operating system. 